# Course Section : AIT614-005
## Lab 2 : NoSQL MongoDB with Python
### Student's Name : Rashmika Calve

In [1]:
!pip install pymongo

#### Importing the required libraries

In [2]:
import pymongo
import pandas as pd
import json

#### Connect to MongoDB

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

#### Load the csv file

In [4]:
df = pd.read_csv("EmployeeAttrition.csv")
df.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


In [5]:
df.shape

(1470, 35)

#### Converting the dataframe to JSON format and Loading to MongoDB

In [6]:
# Converting the df to json 
emp_data = json.loads(df.to_json(orient='records'))

#### Creating a Database

In [7]:
mongo_db = client["myDB"]
mongo_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'myDB')

#### Creating a collection

In [8]:
collection_nm = "Empl_Attrition"

In [9]:
collection_nm = mongo_db[collection_nm]

#### Insert the data into MongoDB Collection

In [10]:
collection_nm.insert_many(emp_data)

#### Query MongoDB

##### Count the total no of documents in the collection

In [11]:
collection_nm.count_documents({})

1470

##### 1. Count the no of employees whose TotalWorkingYears are greater than 20.

In [12]:
collection_nm.count_documents({
    "TotalWorkingYears" : {"$gt" : 20}  })

207

##### 2. Find EmployeeNumber, EducationField, JobRole for all the employees whose Age is between 25 and 30 and Education is 5. Display only EmployeeNumber, EducationField, and JobRobe in the output.

In [13]:
res = collection_nm.find({'Age' : {'$gte' : 25, '$lte' : 30},
                         "Education" : 5},
                         {'EmployeeNumber', 'EducationField','JobRole'}
                        )
print('EmployeeNumber', '\t', 'EducationField', '\t\t', 'JobRole')
print('--------------------------------------------------------')
for r in res:
    print(r['EmployeeNumber'], '\t\t', r['JobRole'], '\t\t', r['EducationField'])



EmployeeNumber 	 EducationField 		 JobRole
--------------------------------------------------------
455 		 Laboratory Technician 		 Other
565 		 Research Scientist 		 Technical Degree
747 		 Sales Executive 		 Marketing
1094 		 Laboratory Technician 		 Life Sciences


##### 3. For all the women employees having Age between 35 and 40 and TotalWorkingYears < 5, sort EmployeeNumber in an ascending order. Print only Department and EmployeeNumber in the output.

In [14]:
# Adding conditions to the find function
emp_res= collection_nm.find(
    {"$and": [
        {"Gender" : 'Female'},
        {'Age' : {'$gte' : 35}},
        {'Age' : {'$lte' : 40}},
        {'TotalWorkingYears' : {'$lt':5}}
    ]},
    {'EmployeeNumber','Department'}    
).sort('EmployeeNumber',1)

In [15]:
# Converting the cursor to a list
emp_res_list = list(emp_res)

In [16]:
# Converting the list to a dataframe
emp_df_3 = pd.DataFrame(emp_res_list)
emp_df_3.shape

(9, 3)

In [17]:
#Displaying the results
emp_df_3[['EmployeeNumber','Department']]

,EmployeeNumber,Department
0,49,Sales
1,75,Research & Development
2,245,Research & Development
3,805,Sales
4,1569,Research & Development
5,1662,Research & Development
6,1675,Research & Development
7,1886,Research & Development
8,2052,Research & Development


##### 4. Find employees whose HourlyRate is greater than or equal to 100 or DailyRate is greater than 1490. Display Age, HourlyRate, DailyRate, and Department only and sort DailyRate in an ascending order.

In [18]:
# Adding conditions to the find function
emp_res4 = collection_nm.find(
    {'$or': [
        {'HourlyRate' : {'$gte':100}},
        {'DailyRate': {'$gt':1490}}
    ]},
    {'Age','HourlyRate','DailyRate','Department'}
).sort('DailyRate',1) # 1 means ascending order


In [19]:
# Converting the cursor to a list
emp_res_list4 = list(emp_res4)
emp_res_list4

[{'_id': ObjectId('63f5348794ccd61cbfa4e495'),
  'Age': 31,
  'DailyRate': 218,
  'Department': 'Sales',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e79b'),
  'Age': 29,
  'DailyRate': 224,
  'Department': 'Research & Development',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e55e'),
  'Age': 45,
  'DailyRate': 306,
  'Department': 'Sales',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e911'),
  'Age': 38,
  'DailyRate': 345,
  'Department': 'Sales',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e6a9'),
  'Age': 35,
  'DailyRate': 528,
  'Department': 'Human Resources',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e3e6'),
  'Age': 22,
  'DailyRate': 594,
  'Department': 'Research & Development',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e411'),
  'Age': 19,
  'DailyRate': 602,
  'Department': 'Sales',
  'HourlyRate': 100},
 {'_id': ObjectId('63f5348794ccd61cbfa4e752'),
  'Age': 26,
  'Dail

In [20]:
# Converting the list to a dataframe
emp_df_4 = pd.DataFrame(emp_res_list4)
emp_df_4.shape

(27, 5)

In [21]:
#Displaying the results
emp_df_4.loc[ : , emp_df_4.columns != '_id']

,Age,DailyRate,Department,HourlyRate
0,31,218,Sales,100
1,29,224,Research & Development,100
2,45,306,Sales,100
3,38,345,Sales,100
4,35,528,Human Resources,100
5,22,594,Research & Development,100
6,19,602,Sales,100
7,26,652,Research & Development,100
8,34,702,Research & Development,100
9,32,976,Sales,100


##### 5. For each JobRole, find the average MonthlyIncome. Print out the formatted monthly incomes in hundredth and arrange them in descending order.

In [22]:
emp_res5 = collection_nm.aggregate([
    {"$group": {
        "_id" : "$JobRole",
        "avg_monthly_income" : {"$avg" : '$MonthlyIncome'}
    }},
    {"$sort" : {
        "avg_monthly_income" : -1 }
    }
])


#Printing the results
print('Job Role', '\t\t\t\t', 'Average Monthly Income')
print('-----------------------------------------------------------------')
for r in emp_res5:
    print(f"{r['_id'] : <25}{'{:.2f}' : >30}".format(r['avg_monthly_income']))

Job Role 				 Average Monthly Income
-----------------------------------------------------------------
Manager                                          17181.68
Research Director                                16033.55
Healthcare Representative                        7528.76
Manufacturing Director                           7295.14
Sales Executive                                  6924.28
Human Resources                                  4235.75
Research Scientist                               3239.97
Laboratory Technician                            3237.17
Sales Representative                             2626.00


##### 6. Count the different MaritalStatus when Attrition is YES and AGE is greater than 35 in the dataset. Arrange the count in descending order.

In [23]:
emp_res6 = collection_nm.aggregate([
    {
        '$match' : {
            '$and': [
                {"Attrition":'Yes'},
                {'Age': {'$gt' : 35}}
            ]
        }
    },
    {
        "$group" : {
            "_id" : "$MaritalStatus",
            "count_emp" : {"$sum" : 1}
        }
    },
    {"$sort" : {
        "count_emp" : -1 }
    }
])

In [24]:
list(emp_res6)

[{'_id': 'Married', 'count_emp': 33},
 {'_id': 'Single', 'count_emp': 30},
 {'_id': 'Divorced', 'count_emp': 14}]

##### Delete All Documents in a Collection

In [25]:
collection_del = collection_nm.delete_many({})

##### Delete the Collection

In [26]:
collection_nm.drop()

#### References

###### [1] Dr. Liao’s lab tutorials and code examples: Blackboard/Liao_PyMongo.html  
###### [2] Python MongoDB - https://www.w3schools.com/python/python_mongodb_getstarted.asp
###### [3] PyMongoDB Documentation - https://pymongo.readthedocs.io/en/stable/